# IAQF Backtester

Input is n x 3 DataFrame where column 1 is the long ETF and column 2 is the short ETF. Column 3 is the signal (1 corresponds to buy and -1 corresponds to sell, there is no reversing)

In [1]:
import pandas as pd 
import numpy as np

In [119]:
filename ='sample_data.csv'
sample_data = pd.read_csv(filename)
sample_data['Date'] = pd.to_datetime(sample_data['Date'])
sample_data = sample_data.set_index('Date')
sample_data = sample_data.dropna()

sample_data.head()

,Stock 1,Stock 2,Signal
Date,,,
2023-02-25,50.0,70.0,1.0
2023-02-26,55.0,65.0,-1.0


- Need way to calculate position size 

In [126]:
def backtest(sample_data):
    data = sample_data.copy()
    cur_pos = (0,0) 
    init_val, prev_val = 0, 0 
    closed = False 
    for i in data.index:
        cur_prices = [data.loc[i,'Stock 1'], data.loc[i,'Stock 2']]
        if data.loc[i,'Signal'] == 1:
            if cur_pos[0] == 0 and cur_pos[1] == 0: 
                init_val = np.dot(cur_pos,cur_prices)
                new_pos = True 
            elif cur_pos[0] < 0 and cur_pos[1] > 0:
                closed = True 
        elif data.loc[i,'Signal'] == -1:
            if cur_pos[0] == 0 and cur_pos[1] == 0:
                cur_pos = (-1,1)
                init_val = np.dot(cur_pos,cur_prices)
                new_pos = True 
            elif cur_pos[0] > 0 and cur_pos[1] < 0:
                closed = True

        if new_pos == True:
            prev_val = init_val
            new_pos = False 

        data.loc[i,'Daily PnL'] = np.dot(cur_pos,cur_prices) - prev_val
        prev_val = np.dot(cur_pos,cur_prices)

        if closed == True:
            cur_pos = (0,0)
            init_val, prev_val = 0, 0
            closed = False 

        data.loc[i,'Pos 1'] = cur_pos[0]
        data.loc[i,'Pos 2'] = cur_pos[1]
        data.loc[i,'Position Value'] = np.dot(cur_pos,cur_prices)

    data['Total PnL'] = data['Daily PnL'].cumsum()
    return data 

In [127]:
backtest(sample_data)

,Stock 1,Stock 2,Signal,Daily PnL,Pos 1,Pos 2,Position Value,Total PnL
Date,,,,,,,,
2023-02-25,50.0,70.0,1.0,0.0,1.0,-1.0,-20.0,0.0
2023-02-26,55.0,65.0,-1.0,10.0,0.0,0.0,0.0,10.0
